In [1]:
import pandas as pd

In [27]:
contrast_df = pd.read_json('./output/eval_predictions_contrast1.jsonl', lines=True)
contrast_df.head()

,id,title,context,question,answers,predicted_answer
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won the 50 super bowl?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,In what city was the stadium where the 50 supe...,"{'text': ['Santa Clara', 'Santa Clara, Califor...","San Francisco Bay Area at Santa Clara, California"
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,What was suspended for the Super Bowl 50?,{'text': ['naming each Super Bowl game with Ro...,the tradition of naming each Super Bowl game w...
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,What team was the champion in Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used in the Super Bowl?,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",Arabic


In [30]:
original_df = pd.read_json('./output/eval_predictions.jsonl', lines=True).head(100)
original_df.head()

,id,title,context,question,answers,predicted_answer
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the NFC at Super Bo...,"{'text': ['Carolina Panthers', 'Carolina Panth...",Carolina Panthers
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,Where did Super Bowl 50 take place?,"{'text': ['Santa Clara, California', 'Levi's S...",Levi's Stadium in the San Francisco Bay Area a...
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team won Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...","golden anniversary"" with various gold"


In [31]:
contrast_df.to_csv('./output/contrast_sets.csv', index=True)
original_df.to_csv('./output/original_sets.csv', index=True)

In [32]:
same  = 0
different = 0

for i in range(100):
    contrast_question = contrast_df.iloc[i]['question']
    original_question = original_df.iloc[i]['question']
    if contrast_question == original_question:
        same += 1
    else:
        different += 1
print(same, different)

91 9


In [33]:
original_df = original_df.head(25)
# drop the col predicted answer
original_df = original_df.drop(['predicted_answer'], axis=1)

original_df.to_csv('./output/squad_contrast.csv', index=True)


In [36]:
squad_contrast = pd.read_csv('./output/squad_contrast.csv', index_col=0)
squad_contrast.head()

,id,title,context,question,answers,perturbation_type
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team did not represent the AFC at Su...,"{'text': [‘Carolina Panthers'], 'answer_start'...",Negation Introduction
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,"Which NFL team, known for their blue and black...","{'text': ['Carolina Panthers', 'Carolina Panth...",Addition of Distractors
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,"In which city, located in the San Francisco Ba...","{'text': ['Santa Clara, California'], 'answer_...",Contextual Reordering
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Who emerged as the champions of Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Paraphrasing
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What theme was used to highlight the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",Question Focus Change


In [38]:
squad_contrast['perturbation_type'].value_counts()

perturbation_type
Paraphrasing                           7
Temporal Perturbation                  4
Question Focus Change                  3
Negation Introduction                  3
Lexical Substitution                   3
Addition of Distractors                2
Contextual Reordering                  2
Coreference Resolution Perturbation    1
Name: count, dtype: int64

In [48]:
# replace \u2018 and \u2019 with '
squad_contrast['answers'] = squad_contrast['answers'].str.replace('\u2018', "'")
squad_contrast['answers'] = squad_contrast['answers'].str.replace('\u2019', "'")

# remove " from answers
squad_contrast['answers'] = squad_contrast['answers'].str.replace('"', '')


# convert to jsonl
squad_contrast.to_json('squad_contrast_detailed.jsonl', orient='records', lines=True)
squad_contrast.drop(['perturbation_type'], axis=1).to_json('squad_contrast.jsonl', orient='records', lines=True)


In [49]:
# test that each 'answers' is valid json
import json
for i in range(100):
    try:
        json.loads(squad_contrast.iloc[i]['answers'])
    except Exception as e:
        print(i)
        print(e)
        print(squad_contrast.iloc[i]['answers'])
        break

0
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
{'text': ['Carolina Panthers'], 'answer_start': [177]}


In [56]:
# load squad_contrast.jsonl, and for each line, remove all double quotes after "answers:"

def process_jsonl_file(input_file, output_file):
    """
    Processes a JSONL file by replacing single quotes with double quotes
    only after 'answer:' and removing unnecessary double quotes after 'answer:'.

    Args:
        input_file (str): Path to the input JSONL file.
        output_file (str): Path to the output JSONL file.
    """
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if '"answers":' in line:
                # Split the line into before and after "answer:"
                before, after = line.split('"answers":', 1)
                
                # Replace single quotes with double quotes in the after part
                after = after.replace("'", '"')
                
                # Optionally, remove double quotes immediately after "answer:"
                if after.startswith('"'):
                    after = after[1:]  # Remove the first double quote
                
                if after[-3] == '"':
                    # remove the double quote
                    after = after[:-3] + after[-2:]

                # Reconstruct the line
                line = before + '"answers":' + after
            
            # Write the processed line to the output file
            outfile.write(line)

# Example usage
input_file = "squad_contrast.jsonl"
output_file = "squad_contrast_fixed.jsonl"
process_jsonl_file(input_file, output_file)


In [2]:
import pandas as pd

contrast_preds = pd.read_json('./output/eval_predictions_contrast25.jsonl', lines=True)
contrast_preds.head()


,id,title,context,question,answers,predicted_answer
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team did not represent the AFC at Su...,"{'text': ['Carolina Panthers'], 'answer_start'...",Denver Broncos
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,"Which NFL team, known for their blue and black...","{'text': ['Carolina Panthers', 'Carolina Panth...",Carolina Panthers
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,"In which city, located in the San Francisco Ba...","{'text': ['Santa Clara, California'], 'answer_...",Santa Clara
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Who emerged as the champions of Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What theme was used to highlight the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",gold


In [3]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

def compute_f1_score(prediction, ground_truth):
    """
    Compute the F1 token-wise score between a prediction and a ground truth answer.
    """
    pred_tokens = prediction.split()
    ground_tokens = ground_truth.split()
    
    common_tokens = set(pred_tokens) & set(ground_tokens)
    if not common_tokens:
        return 0.0

    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(ground_tokens)
    if precision + recall == 0:
        return 0.0

    return 2 * (precision * recall) / (precision + recall)

def add_exact_match_and_f1(df):
    """
    Add exact_match and f1_token columns to the DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with 'answers' and 'predicted_answer' columns.

    Returns:
        pd.DataFrame: Updated DataFrame with two new columns.
    """
    exact_match_results = []
    f1_scores = []

    for _, row in df.iterrows():
        predicted_answer = row['predicted_answer']
        possible_answers = row['answers']['text']  # List of possible answers

        # Check for exact match
        exact_match = predicted_answer in possible_answers
        exact_match_results.append(exact_match)

        # Compute F1 token-wise score
        max_f1 = max(
            compute_f1_score(predicted_answer, answer) for answer in possible_answers
        )
        f1_scores.append(max_f1)

    # Add new columns
    df['exact_match'] = exact_match_results
    df['f1_token'] = f1_scores

    return df

# Example usage
# Assuming `contrast_preds` is your DataFrame
contrast_preds = add_exact_match_and_f1(contrast_preds)


In [4]:
contrast_preds.head()


,id,title,context,question,answers,predicted_answer,exact_match,f1_token
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team did not represent the AFC at Su...,"{'text': ['Carolina Panthers'], 'answer_start'...",Denver Broncos,False,0.0
1,56be4db0acb8001400a502ed,Super_Bowl_50,Super Bowl 50 was an American football game to...,"Which NFL team, known for their blue and black...","{'text': ['Carolina Panthers', 'Carolina Panth...",Carolina Panthers,True,1.0
2,56be4db0acb8001400a502ee,Super_Bowl_50,Super Bowl 50 was an American football game to...,"In which city, located in the San Francisco Ba...","{'text': ['Santa Clara, California'], 'answer_...",Santa Clara,False,0.4
3,56be4db0acb8001400a502ef,Super_Bowl_50,Super Bowl 50 was an American football game to...,Who emerged as the champions of Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos,True,1.0
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What theme was used to highlight the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",gold,True,1.0


In [5]:
contrast_details = pd.read_json('squad_contrast_detailed.jsonl', lines=True)
contrast_preds['perturbation_type'] = contrast_details['perturbation_type'].copy()




In [73]:
contrast_preds.groupby(['different_answer'])[['f1_token', 'exact_match']].agg(['mean', 'count'])#.sort_values('mean')

f1_token       exact_match      
                      mean count        mean count
different_answer                                  
False             0.900000    16       0.875    16
True              0.307407     9       0.000     9

In [74]:
# contrast_preds.groupby('perturbation_type')['f1_token'].agg(['mean', 'count']).sort_values('mean')
contrast_preds.groupby(['perturbation_type'])[['f1_token', 'exact_match']].agg(['mean', 'count'])#.sort_values('mean')

f1_token       exact_match      
                                         mean count        mean count
perturbation_type                                                    
Addition of Distractors              1.000000     2    1.000000     2
Contextual Reordering                0.700000     2    0.500000     2
Coreference Resolution Perturbation  1.000000     1    1.000000     1
Lexical Substitution                 0.800000     3    0.666667     3
Negation Introduction                0.222222     3    0.000000     3
Paraphrasing                         0.857143     7    0.857143     7
Question Focus Change                0.666667     3    0.666667     3
Temporal Perturbation                0.425000     4    0.000000     4

In [76]:
noisy_types = [
    'Addition of Distractors',
    'Contextual Reordering',
    'Coreference Resolution Perturbation',
    'Lexical Substiution',
    'Paraphrasing',
]

contrast_preds[contrast_preds['perturbation_type'].isin(noisy_types)][['f1_token', 'exact_match']].agg('mean')

f1_token       0.866667
exact_match    0.833333
dtype: float64

In [39]:
contrast_preds[contrast_preds['perturbation_type']=='Coreference Resolution Perturbation']

,id_x,title_x,context_x,question_x,answers_x,predicted_answer_x,exact_match,f1_token,perturbation_type,id_y,title_y,context_y,question_y,answers_y,predicted_answer_y
11,56beace93aeaaa14008c91df,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which team claimed victory in the 50th iterati...,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos,True,1.0,Coreference Resolution Perturbation,56beace93aeaaa14008c91df,Super_Bowl_50,Super Bowl 50 was an American football game to...,Who won Super Bowl 50?,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos


In [ ]:
origin_preds = pd.read_json('./output/eval_predictions.jsonl', lines=True).head(25)
contrast_preds = pd.merge(contrast_preds, origin_preds[['question', 'answers', 'predicted_answer']], how='left', left_index=True, right_index=True, suffixes=['c', 'o'])
contrast_preds

In [38]:
context = contrast_preds['context_x'].iloc[0]
print(context)

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [63]:
contrast_preds['different_answer'] = contrast_preds['answers_x'] != contrast_preds['answers_y']
contrast_preds.loc[5, 'different_answer'] = False

In [65]:

contrast_preds.groupby(['perturbation_type'])['different_answer'].agg(['mean', 'count']).sort_values('mean')

,mean,count
perturbation_type,,
Addition of Distractors,0.000000,2
Coreference Resolution Perturbation,0.000000,1
Lexical Substitution,0.000000,3
Paraphrasing,0.000000,7
Question Focus Change,0.333333,3
Contextual Reordering,0.500000,2
Negation Introduction,1.000000,3
Temporal Perturbation,1.000000,4


In [68]:
contrast_preds['different_answer'].sum()

np.int64(9)

In [66]:
contrast_preds[contrast_preds['perturbation_type']=='Question Focus Change']

,id_x,title_x,context_x,question_x,answers_x,predicted_answer_x,exact_match,f1_token,perturbation_type,id_y,title_y,context_y,question_y,answers_y,predicted_answer_y,different_answer
4,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What theme was used to highlight the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...",gold,True,1.0,Question Focus Change,56be4db0acb8001400a502f0,Super_Bowl_50,Super Bowl 50 was an American football game to...,What color was used to emphasize the 50th anni...,"{'text': ['gold', 'gold', 'gold'], 'answer_sta...","golden anniversary"" with various gold",False
16,56bf10f43aeaaa14008c94fd,Super_Bowl_50,Super Bowl 50 was an American football game to...,During which Super Bowl did the Denver Broncos...,"{'text': ['Super Bowl 50', '50'], 'answer_star...","February 7, 2016",False,0.0,Question Focus Change,56bf10f43aeaaa14008c94fd,Super_Bowl_50,Super Bowl 50 was an American football game to...,What year did the Denver Broncos secure a Supe...,"{'text': ['2015', '2016', '2015'], 'answer_sta...",2016,True
18,56bf10f43aeaaa14008c94ff,Super_Bowl_50,Super Bowl 50 was an American football game to...,What was the name of the venue that hosted Sup...,"{'text': ['Levi's Stadium', 'Levi's Stadium', ...",Levi's Stadium,True,1.0,Question Focus Change,56bf10f43aeaaa14008c94ff,Super_Bowl_50,Super Bowl 50 was an American football game to...,What stadium did Super Bowl 50 take place in?,"{'text': ['Levi's Stadium', 'Levi's Stadium', ...",Levi's Stadium,False


In [77]:
origin_preds_100 = pd.read_json('./output/eval_predictions.jsonl', lines=True).head(100)
origin_preds_100 = add_exact_match_and_f1(origin_preds_100)



In [87]:
contrast_preds[contrast_preds['f1_token'] == 0.0]

,id_x,title_x,context_x,question_x,answers_x,predicted_answer_x,exact_match,f1_token,perturbation_type,id_y,title_y,context_y,question_y,answers_y,predicted_answer_y,different_answer
0,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team did not represent the AFC at Su...,"{'text': ['Carolina Panthers'], 'answer_start'...",Denver Broncos,False,0.0,Negation Introduction,56be4db0acb8001400a502ec,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which NFL team represented the AFC at Super Bo...,"{'text': ['Denver Broncos', 'Denver Broncos', ...",Denver Broncos,True
5,56be8e613aeaaa14008c90d1,Super_Bowl_50,Super Bowl 50 was an American football game to...,Which concept was highlighted as the central f...,"{'text': ['golden anniversary', 'gold-themed',...",Arabic numerals 50,False,0.0,Paraphrasing,56be8e613aeaaa14008c90d1,Super_Bowl_50,Super Bowl 50 was an American football game to...,What was the theme of Super Bowl 50?,"{'text': ['""golden anniversary""', 'gold-themed...",an American football game,False
8,56bea9923aeaaa14008c91b9,Super_Bowl_50,Super Bowl 50 was an American football game to...,What was not the theme of Super Bowl 50?,"{'text': ['roman numerals'], 'answer_start': [...",golden anniversary,False,0.0,Negation Introduction,56bea9923aeaaa14008c91b9,Super_Bowl_50,Super Bowl 50 was an American football game to...,What was the theme of Super Bowl 50?,"{'text': ['""golden anniversary""', 'gold-themed...",an American football game,True
16,56bf10f43aeaaa14008c94fd,Super_Bowl_50,Super Bowl 50 was an American football game to...,During which Super Bowl did the Denver Broncos...,"{'text': ['Super Bowl 50', '50'], 'answer_star...","February 7, 2016",False,0.0,Question Focus Change,56bf10f43aeaaa14008c94fd,Super_Bowl_50,Super Bowl 50 was an American football game to...,What year did the Denver Broncos secure a Supe...,"{'text': ['2015', '2016', '2015'], 'answer_sta...",2016,True
21,56d20362e7d4791d009025e8,Super_Bowl_50,Super Bowl 50 was an American football game to...,What decade did Super Bowl 50 take place in?,"{'text': ['2010s', '2010'], 'answer_start': [1...",2015,False,0.0,Temporal Perturbation,56d20362e7d4791d009025e8,Super_Bowl_50,Super Bowl 50 was an American football game to...,What year was Super Bowl 50?,"{'text': ['2015', '2016', '2016'], 'answer_sta...",2015,True


In [95]:
contrast_preds.loc[5, 'question_x']

'Which concept was highlighted as the central focus of Super Bowl 50?'

In [86]:
origin_preds_100[origin_preds_100['f1_token'] == 0.0]

,id,title,context,question,answers,predicted_answer,exact_match,f1_token
5,56be8e613aeaaa14008c90d1,Super_Bowl_50,Super Bowl 50 was an American football game to...,What was the theme of Super Bowl 50?,"{'text': ['""golden anniversary""', 'gold-themed...",an American football game,False,0.0
7,56be8e613aeaaa14008c90d3,Super_Bowl_50,Super Bowl 50 was an American football game to...,What is the AFC short for?,"{'text': ['American Football Conference', 'Ame...",Super Bowl 50,False,0.0
8,56bea9923aeaaa14008c91b9,Super_Bowl_50,Super Bowl 50 was an American football game to...,What was the theme of Super Bowl 50?,"{'text': ['""golden anniversary""', 'gold-themed...",an American football game,False,0.0
47,56d2045de7d4791d009025f5,Super_Bowl_50,The Panthers finished the regular season with ...,How many times have the Panthers been in the S...,"{'text': ['2', 'second', 'second'], 'answer_st...",eight,False,0.0
50,56d6017d1c85041400946ec1,Super_Bowl_50,The Panthers finished the regular season with ...,Who did Denver beat in the 2015 AFC Championsh...,"{'text': ['New England Patriots', 'the New Eng...",Arizona Cardinals,False,0.0
89,56beaf5e3aeaaa14008c9201,Super_Bowl_50,"CBS broadcast Super Bowl 50 in the U.S., and c...",Who was the main performer at this year's half...,"{'text': ['Coldplay', 'Coldplay', 'Coldplay'],...",Beyoncé and Bruno Mars,False,0.0


In [96]:
print(origin_preds_100.loc[89, 'context'])
print(origin_preds_100.loc[89, 'question'])

CBS broadcast Super Bowl 50 in the U.S., and charged an average of $5 million for a 30-second commercial during the game. The Super Bowl 50 halftime show was headlined by the British rock group Coldplay with special guest performers Beyoncé and Bruno Mars, who headlined the Super Bowl XLVII and Super Bowl XLVIII halftime shows, respectively. It was the third-most watched U.S. broadcast ever.
Who was the main performer at this year's halftime show?


In [91]:
origin_preds_100[origin_preds_100['question'].str.contains('not')]

,id,title,context,question,answers,predicted_answer,exact_match,f1_token
44,56bf159b3aeaaa14008c950b,Super_Bowl_50,The Panthers finished the regular season with ...,Which team did not get a chance to defend thei...,"{'text': ['New England Patriots', 'the New Eng...",New England Patriots,True,1.0
